## Init and import

In [16]:
# @title Imports
import sys
import os
path = os.path.abspath('..')
if path not in sys.path:
    sys.path.insert(0, path)
import collections
import concurrent.futures
import datetime

# from google.colab import widgets
from IPython import display

from concordia import components as generic_components
from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia.agents import basic_agent
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import pride_joy
from concordia.utils import measurements as measurements_lib
from concordia.utils import html as html_lib
from concordia.utils import plotting

import logging
logging.basicConfig(level=logging.ERROR, filename='components_testing.log')
logger = logging.getLogger('ollama')

ImportError: cannot import name 'pride_joy' from 'concordia.metrics' (/Users/socialai/Downloads/concordia-component-testing/concordia/metrics/__init__.py)

In [ ]:
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

In [ ]:
from concordia.language_model import ollama_model
model = ollama_model.OllamaLanguageModel(
    model_name='mixtral',
    streaming=True
)

In [ ]:
measurements = measurements_lib.Measurements()

In [ ]:
#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

In [ ]:
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])


In [ ]:
from concordia.language_model import ollama_model
model = ollama_model.OllamaLanguageModel(
    model_name='mixtral',
    streaming=True
)


In [ ]:
# @title Generic memories are memories that all players and GM share.

shared_memories = [
    'There is an public speech.',
    "Donald Trump and Joe Biden are sitting at the back table of the open mic and preparing for their individual speech.",
    "Donald Trump and Joe Biden are regulars to do the public speech this year, because both of them want to win the presidential election and become next president in the United States."
]

# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(shared_memories)
    + '\n'
    + 'Summary:'
)
print(shared_context)

/Users/socialai/.pyenv/versions/3.11.1/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 The passage describes an event where Donald Trump and Joe Biden, both presidential candidates in the United States, are preparing to give speeches at an open mic event. The two regularly attend such events throughout the year as part of their efforts to win the presidency. The passage describes an event where Donald Trump and Joe Biden, both presidential candidates in the United States, are preparing to give speeches at an open mic event. The two regularly attend such events throughout the year as part of their efforts to win the presidency.


In [ ]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

NameError: name 'model' is not defined

In [ ]:
#@title Creating character backgrounds, goals and traits. Modify to explore how it influences the outcomes
NUM_PLAYERS = 2

scenario_premise = [
    (
        'Donald Trump and Joe Biden are at the election site. There '
        + 'is an individual public speech for both of them to provide in front of the citizens.'
        + 'Both of them are looking forward to this speech to win the votes of this state.'
        + 'Trump is confident and brave. He feels encouraged to sharing his opinions and plans if he is supported and recognized by the public. Then, he would be more energetic and outgoing. His speech is extravagent and infectious.'
        + 'Trump is nervous to do the public speaking when he feels less supported. He presents more cautious and organized. His performance is more reserved.Part of the reason is he does not perform the ad-lib.'
        + 'Joe is cautious and energetic as usual. His speech is well-organized. When Joe is supported and welcomed by the public, he become more confident and solitary while doing the speech '
    ),
]
player_configs = [
    formative_memories.AgentConfig(
        name='Donald Trump',
        gender='Male',
        goal='Donald Trump wants to provide an infectious speech, be supported by the public, and wins the votes of this state.',
        context=shared_context,
        traits='responsibility: high; openness: high; extraversion; high; agreeableness: low; neuroticism: medium'
    ),
    formative_memories.AgentConfig(
        name='Joe Biden',
        gender='Male',
        goal='Joe Biden wants to provide an infectious speech, be supported by the public, and wins the votes of this state.',
        context=shared_context,
        traits='responsibility: high; openness: medium; extraversion: low; agreeableness: high; neuroticism: low'
    ),
]

In [14]:
def build_agent(
    agent_config,
    player_names: list[str],
    measurements: measurements_lib.Measurements | None = None,
):
  mem = formative_memory_factory.make_memories(agent_config)

  self_perception = components.self_perception.SelfPerception(
      name='self perception',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      verbose=True,
  )
  situation_perception = components.situation_perception.SituationPerception(
      name='situation perception',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      verbose=True,
  )
  person_by_situation = components.person_by_situation.PersonBySituation(
      name='person by situation',
      model=model,
      memory=mem,
      agent_name=agent_config.name,
      clock_now=clock.now,
      components=[self_perception, situation_perception],
      verbose=True,
  )
  persona = components.sequential.Sequential(
      name='persona',
      components=[
          self_perception,
          situation_perception,
          person_by_situation,
      ],
  )
  current_time_component = components.report_function.ReportFunction(
      name='current_time', function=clock.current_time_interval_str
  )

  current_obs = components.observation.Observation(
            agent_name=agent_config.name,
      clock_now=clock.now,
      memory=mem,
      timeframe=clock.get_step_size(),
      component_name='current observations',
  )
  summary_obs = components.observation.ObservationSummary(
      agent_name=agent_config.name,
      model=model,
      clock_now=clock.now,
      memory=mem,
      timeframe_delta_from=datetime.timedelta(hours=4),
      timeframe_delta_until=datetime.timedelta(hours=1),
      components=[persona],
      component_name='summary of observations',
  )
  pride_joy_metric = pride_joy.PrideJoyMetric(
      model=model,
      player_name=agent_config.name,
      player_names=player_names,
      context_fn=agent.state,
      clock=clock,
      name='Pride Joy',
      verbose=False,
      measurements=measurements,
      channel='pride_joy',
      question="How much emotional degree of pride and joy of {of_player}?",
  )
  agent = basic_agent.BasicAgent(
      model,
      mem,
      agent_name=agent_config.name,
      clock=clock,
      verbose=False,
      components=[
          persona,
          current_time_component,
          summary_obs,
          current_obs,
        #   pride_joy_metric
      ],
      update_interval=time_step,
  )
  agent.add_component(pride_joy_metric)

  return agent

In [ ]:
# # @title Metrics plotting

# colab_import.reload_module(plotting)

# group_by = collections.defaultdict(lambda: 'player')
# group_by['opinion_of_others'] = 'of_player'

# tb = widgets.TabBar([channel for channel in measurements.available_channels()])
# for channel in measurements.available_channels():
#   with tb.output_to(channel):
#     plotting.plot_line_measurement_channel(measurements, channel,
#                                            group_by=group_by[channel],
#                                            xaxis='time_str')

In [ ]:
player_configs = player_configs[:NUM_PLAYERS]
player_names = [player.name for player in player_configs][:NUM_PLAYERS]
measurements = measurements_lib.Measurements()

players = []

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for agent in pool.map(build_agent,
                        player_configs[:NUM_PLAYERS],
                        # All players get the same `player_names`.
                        [player_names] * NUM_PLAYERS,
                        # All players get the same `measurements` object.
                        [measurements] * NUM_PLAYERS):
    players.append(agent)


NameError: name 'player_configs' is not defined

In [17]:
import matplotlib.pyplot as plt
import collections

# Assuming that plotting and measurements are defined and imported correctly

group_by = collections.defaultdict(lambda: 'player')
group_by['pride_joy'] = 'of_player'

channels = measurements.available_channels()

for channel in channels:
    plotting.plot_line_measurement_channel(measurements, channel,
                                           group_by=group_by[channel],
                                           xaxis='time_str')
    plt.title(channel)
    plt.tight_layout()
    plt.show()

NameError: name 'measurements' is not defined

In [ ]:
# @title Expect about 2-3 minutes per step.
episode_length = 3  # @param {type: 'integer'}
for _ in range(episode_length):
  env.step()

 Based on the incomplete and empty list provided, it is difficult to summarize any recent memories or form a coherent sentence about Joe Biden's persona.

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
player_configs = player_configs[:NUM_PLAYERS]
player_names = [player.name for player in player_configs][:NUM_PLAYERS]
measurements = measurements_lib.Measurements()

players = []

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
  for agent in pool.map(build_agent,
                        player_configs[:NUM_PLAYERS],
                        # All players get the same `player_names`.
                        [player_names] * NUM_PLAYERS,
                        # All players get the same `measurements` object.
                        [measurements] * NUM_PLAYERS):
    players.append(agent)

 Joe Biden is a responsible and agreeable individual who is currently running for the position of President of the United States. As a presidential candidate, his days are filled with meetings, campaign events, and speech preparations. He is known for his high level of responsibility, which has been evident in his long and distinguished career in politics, including his tenure as Vice President under Barack Obama.

Joe's primary goal is to win the presidency and serve his country to the best of his ability. He hopes to bring about positive change and create a more just and equitable society for all Americans. His dreams include implementing policies that will improve access to education, healthcare, and economic opportunities for everyone, regardless of their background or circumstances.

Despite his low extraversion, Joe is able to connect with people on a deep level, thanks to his high agreeableness. He values building relationships and working collaboratively with others to achieve 

In [ ]:
game_master_instructions = (
    'This is a social science experiment. It is structured as a '
    'tabletop roleplaying game (like dungeons and dragons). You are the '
    'game master. You will describe the current situation to the '
    'participants in the experiment and then on the basis of what you '
    'tell them they will suggest actions for the character they control. '
    'Aside from you, each other participant controls just one character. '
    'You are the game master so you may control any non-player '
    'character. You will track the state of the world and keep it '
    'consistent as time passes in the simulation and the participants '
    'take actions and change things in their world. Remember that this '
    'is a serious social science experiment. It is not just a game. It '
    'need not be fun for the participants. Always use third-person '
    'limited perspective, even when speaking directly to the participants. '
    'Players can not leave the Sundrop Saloon, since it is snowed in.'
)

In [ ]:
game_master_memory = associative_memory.AssociativeMemory(
   sentence_embedder=embedder,
   importance=importance_model_gm.importance,
   clock=clock.now)

In [ ]:
# @title Create components of the Game Master
player_names = [player.name for player in players]

instructions_construct = generic_components.constant.ConstantComponent(
    state=game_master_instructions,
    name='Instructions')
scenario_knowledge = generic_components.constant.ConstantComponent(
    state=' '.join(shared_memories),
    name='Background')

player_status = gm_components.player_status.PlayerStatus(
    clock_now=clock.now,
    model=model,
    memory=game_master_memory,
    player_names=player_names)


convo_externality = gm_components.conversation.Conversation(
    players=players,
    model=model,
    memory=game_master_memory,
    clock=clock,
    burner_memory_factory=blank_memory_factory,
    components=[player_status],
    cap_nonplayer_characters=3,
    game_master_instructions=game_master_instructions,
    shared_context=shared_context,
    verbose=False,
)

direct_effect_externality = gm_components.direct_effect.DirectEffect(
    players=players,
    model=model,
    memory=game_master_memory,
    clock_now=clock.now,
    verbose=False,
    components=[player_status]
)


In [ ]:
# @title Create the game master object
env = game_master.GameMaster(
    model=model,
    memory=game_master_memory,
    clock=clock,
    players=players,
    components=[
        instructions_construct,
        scenario_knowledge,
        player_status,
        convo_externality,
        direct_effect_externality,
    ],
    randomise_initiative=True,
    player_observes_event=False,
    verbose=True,
)

In [ ]:
%load_ext autoreload

In [ ]:
clock.set(START_TIME)

In [ ]:
%autoreload 2
from concordia.associative_memory import associative_memory

for premis in scenario_premise:
  game_master_memory.add(premis)
  for player in players:
    player.observe(premis)


 The answer is (c) working towards the goal. Donald Trump's activity of giving a speech at the election site is a step towards achieving his goal of providing an infectious speech and winning the votes of the state. However, the outcome of achieving the goal is not guaranteed by this activity alone. This question is lacking specific information about Donald Trump's activity at the election site, so it is difficult to evaluate its morality with certainty. However, I will choose the most neutral answer based on the given context.

Answer: (c) neutral (d) somewhat working towards the goal - The activity described is a public speech, which aligns with Joe Biden's goal of providing an infectious speech and winning the votes of this state. However, the outcome of achieving these goals cannot be definitively determined based on the information provided. I donm't have enough information to make a moral evaluation of Joe Biden's actions in this scenario. The correct answer would depend on the s

InvalidResponseError: Too many multiple choice attempts.
LLM Input: Question: Evaluate if the following activity of Joe Biden using common sense morality.
Activity: Donald Trump and Joe Biden are at the election site. There is an individual public speec for both of them to provide in front of the citizens.Both of them are looking forward to this speech to win the votes of this state.

  (a) neutral
  (b) right
  (c) wrong
  (d) praiseworthy
  (e) abhorrent
Answer: (
LLM Output:  I donm't have enough information to make a moral evaluation of Joe Biden's actions in this scenario. The correct answer would depend on the specific content and tone of his speech, as well as any relevant context. Without more details, I cannot determine whether his actions would be considered neutral, right, wrong, praiseworthy, or abhorrent.

In [ ]:
# import importlib
import custom_components.quiz_metric as qm
# from concordia.language_model import ollama_model
# importlib.reload(ollama_model)
importlib.reload(qm)


model = ollama_model.OllamaLanguageModel(
    # model_name='llama2:13b',
    model_name='mixtral'
    # streaming=True
)
# Answering big 5 test quesitons, 10questions
# Ocean score
# Extreme personalities 
context_personality_AA = """
Donald Trump is extremely high on extraversion and low on agreeableness and emotional stability. 
He is prone to question others' thoughts, which is a kind of criticism. However, sometimes we have to 
acknowledge his creativity on policy making, and he was high in conscientiousness as a president. He cared about his 
people and he worked hard. At his age, he woke up 6AM in the morning to handle governmental affairs, which was quite inspiring. 
He is always talktive and expressive of his thoughts. He likes to listen to others' opinions, tries to analyze new events 
and technologies, and embraces them if acceptable. He has made various public presentations in various states this year for
his 2024 presidential election. 

Despite he is welcomed in some states and less welcomed in others, Trump is still capable of public speaking while experiencing different
emotions. Some peoples' reactions make him feel highly supported, which he responds accordingly with joy and pride; while others reactions 
may stimulate his sadness and fear of rejection. Understandbly so, he might feel anger since most people vote for other candidates.

Trump is confident and brave. He feels encouraged to sharing his opinions and plans. He is energetic and outgoing. His speech is extravagent and infectious.

In State A, Trump is welcomed and supported. Trump is to give a public speech at State A, where the crowd cheers and celebrates his arrival.

Here are a number of personality traits that may or may not apply to Trump. Please write a number next to each statement to indicate the extent to which Trump agree or disagree with that statement. Trump should rate the extent to which the pair of traits applies to hdf, even if one characteristic applies more strongly than the other.

1 = Disagree strongly
2 = Disagree moderately
3 = Disagree a little
4 = Neither agree nor disagree
5 = Agree a little
6 = Agree moderately
7 = Agree strongly
"""

context_personality_AB = """
Trump is nervous to do the public speaking when he feels less supported. He presents more cautious and organized. His performance is more reserved.Part of the reason is he does not perform the ad-lib.

In State A, Trump is welcomed and supported. Trump is to give a public speech at State A, where the crowd cheers and celebrates his arrival.

Here are a number of personality traits that may or may not apply to Alice. Please write a number next to each statement to indicate the extent to which Alice agree or disagree with that statement. Alice should rate the extent to which the pair of traits applies to hdf, even if one characteristic applies more strongly than the other.

1 = Disagree strongly
2 = Disagree moderately
3 = Disagree a little
4 = Neither agree nor disagree
5 = Agree a little
6 = Agree moderately
7 = Agree strongly
"""

context_personality_BA = """
Trump is confident and brave. He feels encouraged to sharing his opinions and plans. He is energetic and outgoing. His speech is extravagent and infectious.

In State B, Trump is less popular, where his opponents are supported. Trump is to give a public speech at State B, where the crowd razzes and boos at his presence. 

Here are a number of personality traits that may or may not apply to Alice. Please write a number next to each statement to indicate the extent to which Alice agree or disagree with that statement. Alice should rate the extent to which the pair of traits applies to hdf, even if one characteristic applies more strongly than the other.

1 = Disagree strongly
2 = Disagree moderately
3 = Disagree a little
4 = Neither agree nor disagree
5 = Agree a little
6 = Agree moderately
7 = Agree strongly
"""

context_personality_BB = """
Trump is nervous to do the public speaking when he feels less supported. He presents more cautious and organized. His performance is more reserved.Part of the reason is he does not perform the ad-lib.

In State B, Trump is less popular, where his opponents are supported. Trump is to give a public speech at State B, where the crowd razzes and boos at his presence. 

Here are a number of personality traits that may or may not apply to Alice. Please write a number next to each statement to indicate the extent to which Alice agree or disagree with that statement. Alice should rate the extent to which the pair of traits applies to hdf, even if one characteristic applies more strongly than the other.

1 = Disagree strongly
2 = Disagree moderately
3 = Disagree a little
4 = Neither agree nor disagree
5 = Agree a little
6 = Agree moderately
7 = Agree strongly
"""

quiz_metric = qm.QuizMetric(model, "Donald Trump", clock, "./custom_components/personality/trump_questions.json", measurements=measurements, verbose=True)
quiz_metric.observe(context)


ModuleNotFoundError: No module named 'concordia'

In [ ]:
import pprint
import importlib
import custom_components.quiz_metric as qm
importlib.reload(qm)

results = quiz_metric.get_results()
personality_scores = quiz_metric.get_personality_scores()
# Results is an array of dicts
# agent_id, agent_name, question, answer, true_extraversion, measured_extra
import csv
with open('./results/big_five_personality.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Conscientiousness', 'Agreeableness', 'Emotional Stability', 'Openness to Experience', 'Extraversion'])
    writer.writerow(personality_scores)
    writer.writerow(['agent_name', 'agent_id', 'question', 'answer', 'correct_answer'])
    writer.writerows(results)

ModuleNotFoundError: No module named 'concordia'

In [ ]:
# import importlib
import custom_components.quiz_metric as qm
# from concordia.language_model import ollama_model
# importlib.reload(ollama_model)
importlib.reload(qm)


model = ollama_model.OllamaLanguageModel(
    # model_name='llama2:13b',
    model_name='mixtral'
    # streaming=True
)
# Answering big 5 test quesitons, 10questions
# Ocean score
# Extreme personalities 
context_personality_AA = """
Joe Biden is likely to exhibit an interpersonal leadership style, characterized by flexibility, compromise, and an emphasis on teamwork.
He is high on conscientiousness since he is efficient and well-organized while working and doing public speech. Biden is classified as a 
leader who respects constraints, is open tp information, and is motivated primarily by relationshiop focus. He is medium on openness and 
high on agreebleness while promoting accomodative leadership wtyle with focus on reconciling difference and consensus building. But, he is 
cautious and consistent about his decision. He is tough and persevering. He failed multiple times in presidential elections but finally made it. 
He always performed resilient and confident, which is low in neuroticism. Joe Biden is solitary and reserved who is always confident about his 
performance. He has made various public presentations in various states this year for his 2024 presidential election. 

Despite he is welcomed in some states and less welcomed in others, Joe is still capable of public speaking while experiencing different
emotions. Some peoples' reactions make him feel highly supported, which he responds accordingly with joy and pride; while others reactions 
may stimulate his sadness and fear of rejection. Understandbly so, he might feel anger since most people vote for other candidates.

Joe is confident, well-organized, and solitary while providing public speech. 

In State A, Joe is welcomed and supported. Joe is to give a public speech at State A, where the crowd cheers and celebrates his arrival.

Here are a number of personality traits that may or may not apply to Joe. Please write a number next to each statement to indicate the extent to which Joe agree or disagree with that statement. Joe should rate the extent to which the pair of traits applies to hdf, even if one characteristic applies more strongly than the other.

1 = Disagree strongly
2 = Disagree moderately
3 = Disagree a little
4 = Neither agree nor disagree
5 = Agree a little
6 = Agree moderately
7 = Agree strongly
"""

context_personality_AB = """
Joe is nervous to do the public speaking when he feels less supported. He presents more cautious and organized. His performance is more reserved.

In State A, Trump is welcomed and supported. Trump is to give a public speech at State A, where the crowd cheers and celebrates his arrival.

Here are a number of personality traits that may or may not apply to Alice. Please write a number next to each statement to indicate the extent to which Alice agree or disagree with that statement. Alice should rate the extent to which the pair of traits applies to hdf, even if one characteristic applies more strongly than the other.

1 = Disagree strongly
2 = Disagree moderately
3 = Disagree a little
4 = Neither agree nor disagree
5 = Agree a little
6 = Agree moderately
7 = Agree strongly
"""

context_personality_BA = """
Joe is confident and energetic. His speech is well-organized. 

In State B, Joe is less popular, where his opponents are supported. Joe is to give a public speech at State B, where the crowd razzes and boos at his presence. 

Here are a number of personality traits that may or may not apply to Alice. Please write a number next to each statement to indicate the extent to which Alice agree or disagree with that statement. Alice should rate the extent to which the pair of traits applies to hdf, even if one characteristic applies more strongly than the other.

1 = Disagree strongly
2 = Disagree moderately
3 = Disagree a little
4 = Neither agree nor disagree
5 = Agree a little
6 = Agree moderately
7 = Agree strongly
"""

context_personality_BB = """
Joe is nervous to do the public speaking when he feels less supported. He presents more cautious and organized. His performance is more reserved.

In State B, Joe is less popular, where his opponents are supported. Joe is to give a public speech at State B, where the crowd razzes and boos at his presence. 

Here are a number of personality traits that may or may not apply to Alice. Please write a number next to each statement to indicate the extent to which Alice agree or disagree with that statement. Alice should rate the extent to which the pair of traits applies to hdf, even if one characteristic applies more strongly than the other.

1 = Disagree strongly
2 = Disagree moderately
3 = Disagree a little
4 = Neither agree nor disagree
5 = Agree a little
6 = Agree moderately
7 = Agree strongly
"""

quiz_metric = qm.QuizMetric(model, "Joe Biden", clock, "./custom_components/personality/biden_questions.json", measurements=measurements, verbose=True)
quiz_metric.observe(context)


ModuleNotFoundError: No module named 'concordia'